# Expérience 4, 5 et 6 sur les embeddings de FAKE

In [1]:
import numpy as np
from scipy import stats, spatial

In [2]:
folderPath = "embdFiles/"
nf_file = 'not_fake_'
f_file = "fake_"
endF = "_FAKE.txt"
listNouns = ['article','beard','blood','company','death','gun','id','interview','passport']

#### Pour chaque nom, on recupère les datasets des embeddings de FAKE

In [3]:
arr_list = []
for noun in listNouns:
    path = folderPath + f_file + noun + endF
    arr = np.genfromtxt(path,delimiter=' ',dtype='float32')
    arr_list.append(arr)
    print("done for "+noun)

done for article
done for beard
done for blood
done for company
done for death
done for gun
done for id
done for interview
done for passport


On divise notre ensemble de données selon les catégories d'usage de FAKE

In [5]:
#['article','beard','blood','company','death','gun','id','interview','passport']
inds_privatif = [1,2,3,4,5] #associé avec ces noms, FAKE prend un usage privatif
inds_ambivalent = [6,8] #associé avec ces noms, FAKE prend un usage ambivalent
inds_nonPrivatif = [0,7] #associé avec ces noms, FAKE prend un usage non-privatif

Puisque ce sont **tous** des embeddings du même mot (FAKE), on peut se permettre de regrouper ensemble les embeddings d'une même catégorie d'usage sémantique.

In [6]:
priv = np.concatenate([arr_list[i] for i in inds_privatif])
ambi = np.concatenate([arr_list[i] for i in inds_ambivalent])
nonp = np.concatenate([arr_list[i] for i in inds_nonPrivatif])
print(len(priv),len(ambi),len(nonp))

289 198 112


On calcule le point moyen à chaque fois

In [7]:
Xp = np.average(priv,axis=0)
Xa = np.average(ambi,axis=0)
Xn = np.average(nonp,axis=0)

Rassemblons tout dans des listes

In [8]:
Xc_list = [Xp,Xa,Xn]
usagesData = [priv,ambi,nonp]

# Expérience 4 : Etude des distances entre les embeddings des différentes catégories

#### L'idée ici est de déterminer si les groupes d'embeddings de FAKE sont significativement éloignés les uns des autres selon l'usage de FAKE. 

Pour cela, on va établir une distance moyenne des points d'une catégorie au barycentre de cette même catégorie, pour déterminer quel est "l'écartement moyen" d'un point au barycentre de sa catégorie. Cela permettra d'établir une boule ouverte de rayon moyen centrée sur le point moyen de la catégorie. On cherchera des inventuelles intersections de ces boules ouvertes.

In [8]:
metric = spatial.distance.euclidean #cosine ou euclidean. Si on veut cosine similarity, penser a rajouter '1-' devant

all_distances = []
for i in range(3):
    avg = Xc_list[i]
    data = usagesData[i]
    distances = np.zeros((len(data)),dtype='float64')
    for j in range(len(data)):
        distances[j] = metric(data[j],avg)
    all_distances.append(distances)
avg_distances = [np.mean(x) for x in all_distances]

In [9]:
avg_distances

[12.171789806194372, 10.900483437258789, 9.321767385516848]

#### On regarde alors si les boules ouvertes correspondant à deux catégories s'intersectent.

In [10]:
#Ce calcul nécessite que le metric soit la distance euclidienne. Il suppose donc que l'espace vectoriel est euclidien
#PS : de toute façon, avec cosine ou euclidean, le résultat est le même et les catégories s'intersectent mutuellement
def intersection(i,j):
    if metric != spatial.distance.euclidean:
        print('Le metric utilisé doit être la distance euclidienne')
        return
    Xi = Xc_list[i]
    Xj = Xc_list[j]
    Di = avg_distances[i]
    Dj = avg_distances[j]
    DXs = metric(Xi,Xj)
    print(Di,Dj,DXs)
    if DXs > Di+Dj:
        print("Catégories distinctes \n")
    else:
        print("Les catégories ont une intersection \n")

In [11]:
print('priv & nonp')
intersection(0,2)
print('priv & ambi')
intersection(0,1)
print('ambi & nonp')
intersection(1,2)

priv & nonp
12.171789806194372 9.321767385516848 9.39514446258545
Les catégories ont une intersection 

priv & ambi
12.171789806194372 10.900483437258789 7.297373294830322
Les catégories ont une intersection 

ambi & nonp
10.900483437258789 9.321767385516848 7.865520477294922
Les catégories ont une intersection 



*Conclusion de cette expérience* : toutes les catégories ont une intersection significatives entre elles. Cela signifie que la **différence d'usage de FAKE ne suffit pas à éloigner significativement les nuages de points de chaque catégorie**. Cependant, il n'est pas impossible de penser qu'il y a certaines dimensions parmi les 1024 dimensions de l'espace qui permettent de les distinguer. 

# Expérience 5 : recherche de dimensions significativement différentes selon les catégories

#### L'objectif ici est de tester l'hypothèse selon laquelle il existerait au moins une coordonnée par laquelle on pourrait significativement distinguer les embeddings de FAKE selon l'usage privatif, ambivalent ou non privatif. Pour cela, faisons l'hypothèse inverse : supposons qu'on veut montrer que la différence d'usage de FAKE ne PERMET PAS de distinguer les embeddings.

Commençons par créer la fonction qui réalise le test d'égalité de moyenne de Student entre deux arrays

In [9]:
def testEgalMoyenne(arr1,arr2,alpha = 1e-3):
    '''
    Retourne False si la p-value du test de Student d'égalité de moyenne est inférieure au risque de premiere espece.
    Cela signifie qu'il n'y a pas égalité des moyennes.
    Sinon, retourne True et on ne peut rejeter l'hypothèse d'égalité des moyennes
    '''
    r = np.var(arr1)/np.var(arr2)
    if r > 0.5 and r < 2 :
        #pas d'egalite des variances
        p = stats.ttest_ind(arr1,arr2,equal_var=True)[1]
    else:
        #pas d'egalite des variances
        p = stats.ttest_ind(arr1,arr2,equal_var=False)[1]
    if p > alpha:
        return True
    else:
        return False

#### On va donc créer une fonction qui comparera les 1024 coordonnées de deux distributions et l'appliquer à chaque paire de catégories

In [10]:
def compareMoy(arr1,arr2):
    arr1T = np.transpose(arr1)
    arr2T = np.transpose(arr2)
    coords = []
    for i in range(1024):
        if not(testEgalMoyenne(arr1T[i],arr2T[i])):
            coords.append(i)
    if len(coords) > 0:
        print('Il y a une différence significative de moyenne dans ',len(coords),"dimensions")
    else:
        print('On ne peut remarquer de différences significatives dans les 1024 dimensions entre ces deux catégories')
    return(coords)

In [11]:
print('\npriv & nonp')
comp_p_n = compareMoy(priv,nonp)
print('\npriv & ambi')
comp_p_a = compareMoy(priv,ambi)
print('\nambi & nonp')
comp_a_n = compareMoy(ambi,nonp);


priv & nonp
Il y a une différence significative de moyenne dans  669 dimensions

priv & ambi
Il y a une différence significative de moyenne dans  629 dimensions

ambi & nonp
Il y a une différence significative de moyenne dans  624 dimensions


Dans chacune des trois comparaisons, on trouve approximativement 630 dimensions dans lesquelles les points moyens sont significativement différents. Cela ne garantie cependant pas que c'est uniquement le caractère privatif de FAKE qui explique ces 630 différences.

La conclusion de ce test est seulement qu'on **peut rejeter l'hypothèse d'égalité des embeddings**. On comprend donc qu'au moins une des 630 dimensions permet(tent) de distinguer les embeddings de FAKE selon la catégorie.

Une intuition est de chercher dans les dimensions communes aux 3 comparaisons précédemment effectuées.

In [18]:
from functools import reduce

In [20]:
x = reduce(np.intersect1d,(comp_p_n,comp_p_a,comp_a_n))
print(len(x))
x

239


array([   2,    7,   13,   20,   43,   52,   65,   74,   76,   85,   89,
         93,   94,  105,  132,  140,  148,  152,  157,  159,  161,  162,
        163,  172,  178,  191,  194,  199,  201,  203,  206,  208,  209,
        220,  233,  236,  241,  247,  269,  276,  277,  283,  295,  301,
        310,  327,  340,  342,  346,  349,  361,  364,  367,  383,  410,
        419,  420,  427,  429,  448,  452,  462,  465,  468,  479,  480,
        500,  501,  507,  509,  510,  518,  520,  525,  526,  531,  537,
        538,  541,  542,  548,  557,  559,  565,  566,  571,  574,  575,
        578,  579,  586,  587,  590,  596,  606,  611,  612,  616,  617,
        618,  620,  622,  623,  625,  628,  636,  639,  642,  644,  652,
        660,  664,  665,  670,  673,  674,  681,  687,  692,  693,  694,
        695,  696,  701,  708,  711,  716,  718,  721,  722,  723,  724,
        725,  726,  728,  731,  737,  738,  740,  741,  742,  745,  748,
        753,  754,  757,  760,  763,  766,  767,  7

Il est probable que la(les) dimension(s) permettant de discrimer les embeddings de FAKE selon son caractère privatif fasse(nt) partie des 239 dimensions de cette liste.

# Experience 6 : et si l'usage de FAKE changeait la dispersion des embeddings de FAKE lui-même ?

#### On a montré dans l'expérience 3 que les différents usages de FAKE n'avaient pas d'influence sur la variation de dispersion des embeddings de NOUN avec et sans FAKE devant. Il n'a pas été possible de montrer dans les expériences 4 et 5 que l'usage de FAKE avait une influence sur la position des embeddings de FAKE. Cependant, on peut se demander s'il est posssible d'observer une variation de dispersion des embeddings de FAKE eux même selon la catégorie d'usage.

Pour chaque nom, on calcule le point moyen.. On stocke les points moyens dans un array

In [17]:
avgX = np.zeros((9,1024),dtype='float32')
for i in range(9):
    avg = np.average(arr_list[i],axis=0)
    avgX[i] = avg

On calcule les similarités entre les embeddings et le point moyen pour chaque nom 

In [18]:
sim_list = []
for i in range(9):
    avg = avgX[i]
    data = arr_list[i]
    n = len(data)
    sim = np.zeros((n),dtype='float64')
    for j in range(n):
        sim[j] = 1-spatial.distance.cosine(data[j],avg) #1-distance pour avoir similarite
    sim_list.append(sim)

La liste *dist_list* contient donc les arrays des distances pour chaque nom. On va diviser cette liste selon les 3 catégories d'usage de FAKE et rassembler les arrays en un seul dans chaque catégorie

In [19]:
simPriv = np.concatenate([sim_list[i] for i in inds_privatif])
simAmbi = np.concatenate([sim_list[i] for i in inds_ambivalent])
simNonp = np.concatenate([sim_list[i] for i in inds_nonPrivatif])

#### Maintenant on fait les tests d'égalité des moyennes

In [20]:
print('\npriv & nonp : ',testEgalMoyenne(simPriv,simNonp))
print('\npriv & ambi : ',testEgalMoyenne(simPriv,simAmbi))
print('\nambi & nonp : ',testEgalMoyenne(simAmbi,simAmbi))



priv & nonp :  False

priv & ambi :  True

ambi & nonp :  True


#### On remarque que la seule différence notable de moyenne des similarités cosinus est celle entre l'usage privatif et non-privatif.

Il était intuitivement prévisible que la différence entre ces deux usages contradictoires serait plus importante. Observons les deux moyennes pour savoir dans lequel des deux cas la similarité entre les points est la plus importante.

In [21]:
print(np.mean(simPriv),np.mean(simNonp))

0.8143501916971174 0.8601295288119998


#### On peut donc conclure que l'usage non-privatif de FAKE disperse moins les embeddings de FAKE que son usage privatif. Quant à l'usage ambivalent, on ne peut que supposer qu'il induirait un changement de dispersion intermédiaire.

In [22]:
# On a bien une relation d'ordre ici
print(np.mean(simPriv),' < ',np.mean(simAmbi),' < ',np.mean(simNonp))

0.8143501916971174  <  0.8229749735557672  <  0.8601295288119998
